In [11]:
from crawlbase import CrawlingAPI

# Initialize the CrawlingAPI class with your Crawlbase API token
api = CrawlingAPI({ 'token': '6RJSQVewe3WOG8n6VdeiCQ' })

# options for Crawling API
options = {
 'page_wait': 5000,
 'ajax_wait': 'true'
}

# Construct the request URL
page_url = 'https://www.airbnb.com/s/Belo-Horizonte--MG/homes?flexible_trip_lengths%5B%5D=one_week&monthly_start_date=2025-03-01&monthly_length=3&monthly_end_date=2025-06-01&adults=0&place_id=ChIJMyzPysqQpgARlznSOl55NVs&refinement_paths%5B%5D=%2Fhomes&location_bb=wZ415cIvdYvBoDuiwjBA1Q%3D%3D&acp_id=8949c462-3712-48e3-8bed-185932a32fe1&date_picker_type=calendar&source=structured_search_input_header&search_type=autocomplete_click'

# Make a GET request to the Airbnb search page
response = api.get(page_url, options)

# Check if the request was successful (status code 200)
if response['status_code'] == 200:
    # Extracted HTML content after decoding byte data
    page_html = response['body'].decode('latin1')
    print(page_html)
else:
    print("Failed to retrieve the page. Status code:", response['status_code'])

KeyboardInterrupt: 

In [10]:
# Export the page HTML to a file for later analysis
with open('airbnb_page.html', 'w', encoding='utf-8') as f:
    f.write(page_html)
print("Page HTML has been saved to airbnb_page.html")


Page HTML has been saved to airbnb_page.html


In [9]:
#Import necessary libraries
import json
from bs4 import BeautifulSoup
from crawlbase import CrawlingAPI

# Initialize the CrawlingAPI class with your Crawlbase API token
api = CrawlingAPI({ 'token': '6RJSQVewe3WOG8n6VdeiCQ' })

# options for Crawling API
options = {
 'page_wait': 5000,
 'ajax_wait': 'true'
}

def scrape_page(page_url):
    try:
        # Make a GET request to the Airbnb page
        response = api.get(page_url, options)

        # Check if the request was successful (status code 200)
        if response['status_code'] == 200:
            # Extracted HTML content after decoding byte data
            page_html = response['body'].decode('latin1')

            # Find the JSON data in the script tag
            soup = BeautifulSoup(page_html, 'html.parser')
            scripts = soup.find_all('script')
            
            experiences_data = None
            for script in scripts:
                if script.string and 'ExperienceSearchResult' in script.string:
                    # Extract the JSON data
                    start = script.string.find('[{"__typename":"ExperienceSearchResult"')
                    end = script.string.find(']', start) + 1
                    if start != -1 and end != -1:
                        json_str = script.string[start:end]
                        experiences_data = json.loads(json_str)
                        break

            if experiences_data:
                # Process and return the experiences data
                processed_experiences = []
                for exp in experiences_data:
                    experience = {
                        'title': exp.get('title', ''),
                        'price': exp.get('displayPrice', {}).get('primaryLine', {}).get('price', ''),
                        'duration': exp.get('kickerText', ''),
                        'rating': exp.get('avgRatingLocalized', ''),
                        'location': {
                            'lat': exp.get('lat', ''),
                            'lng': exp.get('lng', '')
                        }
                    }
                    processed_experiences.append(experience)
                return processed_experiences

        return []

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def main():
    page_url = 'https://www.airbnb.com/s/Belo-Horizonte--MG/experiences'
    experiences = scrape_page(page_url)

    # Print or save all experiences
    print(json.dumps(experiences, indent=2))

if __name__ == "__main__":
    main()

An error occurred: <urlopen error [Errno -3] Temporary failure in name resolution>
[]
